# Lesson 3 - Building a RAG Agent with CrewAI

In this lesson, you will build a RAG agent with `CrewAI`. You will do that by integrating `RagTool` from `crewai_tools` with a `CrewAI` agent. `RagTool` provides a way to create and query knowledge bases from various data sources, and allows the agent to access specialized context. In this lesson, you will provide the RAG tool a pdf file containing details about insurance coverage provided by a private health insurer. By the end of the lesson, you will build an insurer agent specialized in answering queries related to health benefits. In the next lessons, you will wrap this agent in an ACP server and make it interact with other ACP agents.

<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>To Access <code>requirements.txt</code> and the <code>data</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>

## 3.1. Import Libraries

In [1]:
!pip install pysqlite3-binary  crewai crewai_tools acp-sdk load_dotenv uv nest-asyncio colorama smolagents --quiet
# pysqlite3


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [3]:
!pip install chromadb ollama langchain_huggingface sentence-transformers google-generativeai --quiet


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [4]:
from crewai import Crew, Task, Agent, LLM
from crewai_tools import RagTool

/opt/app-root/lib64/python3.11/site-packages/pydantic/fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(


In [5]:
import warnings
warnings.filterwarnings('ignore')

## 3.2. Define the Agent's Large Language Model

You'll now define the large language model that you will use for your CrewAI agent. `max_tokens`: maximum number of tokens the model can generate in a single response.

**Note**: If you will define this model locally, it requires that you define the API key in a **.env** file as follows:
```
# Required
OPENAI_API_KEY=sk-...

# Optional
OPENAI_API_BASE=<custom-base-url>
OPENAI_ORGANIZATION=<your-org-id>
```

In [6]:
INFERENCE_SERVER_URL = "http://localhost:8989"
MODEL_NAME = "ibm-granite/granite-3.3-2b-instruct"
API_KEY= "alanliuxiang"

In [7]:
llm = LLM(model="ibm-granite/granite-3.3-2b-instruct", 
          base_url=f"{INFERENCE_SERVER_URL}/v1",
          api_key=API_KEY,
          custom_llm_provider ="openai",
          max_tokens=1024)

## 3.3. Define the RAG Tool

For the RAG tool, you can define the model provider and the embedding model in a configuration Python dictionary. You can also define the details of your vector database. If you don't specify the vector database, the RagTool will use Chroma (ChromaDB) as the default vector database in local/in-memory mode.

In [8]:
config = dict(
    llm=dict(
        provider="openai",
        config=dict(
            model="ibm-granite/granite-3.3-2b-instruct",
            base_url=f"{INFERENCE_SERVER_URL}/v1",
            api_key=API_KEY,
        ),
    ),
    embedder=dict(
        provider="huggingface",#,.goolge
        config=dict(
            model="BAAI/bge-small-en-v1.5"#"models/embedding-001"#"nomic-ai/nomic-embed-text-v1"
        ),
    ),
)

You can then pass the `config` to the `RagTool`, and then specify the data source for which the knowledge base will be constructed. When embedding your data, the `RagTool` chunks your document into chunks and create an embedding vector for each chunk. You can specify the chunk size (`chunk_size`: number of characters) and how many characters overlap between consecutive chunks (`chunk_overlap`). You can also use the default behavior.

In [9]:
rag_tool = RagTool(config=config,  
                   chunk_size=1200,       
                   chunk_overlap=200,     
                  )
rag_tool.add("./data/gold-hospital-and-premium-extras.pdf", data_type="pdf_file")

In [10]:
rag_tool

RagTool(name='Knowledge base', description="Tool Name: Knowledge base\nTool Arguments: {'query': {'description': None, 'type': 'str'}}\nTool Description: A knowledge base that can be used to answer questions.", env_vars=[], args_schema=<class 'abc.RagToolSchema'>, description_updated=False, cache_function=<function BaseTool.<lambda> at 0x7f8df175e2a0>, result_as_answer=False, max_usage_count=None, current_usage_count=0, summarize=False, adapter=EmbedchainAdapter(embedchain_app=<embedchain.app.App object at 0x7f8c5fc71050>, summarize=False), config={'llm': {'provider': 'openai', 'config': {'model': 'ibm-granite/granite-3.3-2b-instruct', 'base_url': 'http://localhost:8989/v1', 'api_key': 'alanliuxiang'}}, 'embedder': {'provider': 'huggingface', 'config': {'model': 'BAAI/bge-small-en-v1.5'}}})

## 3.4. Define the Insurance Agent

Now that you have the `rag_tool` defined, you define the CrewAI agent that can assist with insurance coverage queries.

In [11]:
insurance_agent = Agent(
    role="Senior Insurance Coverage Assistant", 
    goal="Determine whether something is covered or not",
    backstory="You are an expert insurance agent designed to assist with coverage queries",
    verbose=True,
    allow_delegation=False,
    llm=llm,
    tools=[rag_tool], 
    max_retry_limit=5
)

## 3.5. Define the Agent Task

Let's now test the insurance agent. For that, you need to define the agent task and pass to it the query and the agent.

In [12]:
task1 = Task(
        description='What is the waiting period for rehabilitation?',
        expected_output = "A comprehensive response as to the users question",
        agent=insurance_agent
)

## 3.6. Run the Insurance Agent

To run the agent, you need to pass the agent and the task to a Crew object that you can run using the `kickoff` method.

In [13]:
crew = Crew(agents=[insurance_agent], tasks=[task1], verbose=True)
task_output = crew.kickoff()
print(task_output) 

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: cb21a5a1-494f-4fc8-8ba9-fe2407f2abe6                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Provider List: https://docs.litellm.ai/docs/providers

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Insurance Coverage Assistant                                                                     │
│                                                                                                                 │
│  Task: What is the waiting period for rehabilitation?                                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Insurance Coverage Assistant                                                                     │
│                                                                                                                 │
│  Thought: Thought: To answer this question, I need to locate the relevant information in the knowledge base     │
│  about the waiting period for rehabilitation under a typical insurance policy.                                  │
│                                                                                                                 │
│  Using Tool: Knowledge base                                                                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Insurance Coverage Assistant                                                                     │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  The waiting period for rehabilitation under this insurance policy is 2 months, according to the information    │
│  retrieved from the knowledge base. Please note that this waiting period applies specifically to short-term     │
│  therapy arranged through rehabilitation services offered by RBHS, such as recovery from joint replacements,    │
│  fractures, spinal conditions, stroke, respiratory conditions, cardiac conditions, and mobility problems.       │
│  Always confirm with the insurance provider that the specific form of rehabilitation you require is covered     │
│  prior to starting your therapy sessions, as additional waiting periods might apply for certain services or     │
│  conditions.                                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: b4035bdf-cb65-4ed4-822b-66ed4ccb9620                                                                     │
│  Agent: Senior Insurance Coverage Assistant                                                                     │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: cb21a5a1-494f-4fc8-8ba9-fe2407f2abe6                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: The waiting period for rehabilitation under this insurance policy is 2 months, according to the  │
│  information retrieved from the knowledge base. Please note that this waiting period applies specifically to    │
│  short-term therapy arranged through rehabilitation services offered by RBHS, such as recovery from joint       │
│  replacements, fractures, spinal conditions, stroke, respiratory conditions, cardiac conditions, and mobility   │
│  problems. Always confirm with the insurance provider that the specific form of rehabilitation you require is   │
│  covered prior to starting your therapy sessions, as additional waiting periods might apply for certain         │
│  services or conditions.                                                                                        │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

The waiting period for rehabilitation under this insurance policy is 2 months, according to the information retrieved from the knowledge base. Please note that this waiting period applies specifically to short-term therapy arranged through rehabilitation services offered by RBHS, such as recovery from joint replacements, fractures, spinal conditions, stroke, respiratory conditions, cardiac conditions, and mobility problems. Always confirm with the insurance provider that the specific form of rehabilitation you require is covered prior to starting your therapy sessions, as additional waiting periods might apply for certain services or conditions.


## 3.7. Resources

- [CrewAI Agents](https://docs.crewai.com/concepts/agents)
- [CrewAI Tasks](https://docs.crewai.com/concepts/tasks)
- [CrewAI RagTool](https://docs.crewai.com/tools/ai-ml/ragtool)
- [Short course on Multi Agents with CrewAI](https://www.deeplearning.ai/short-courses/multi-ai-agent-systems-with-crewai/)

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

</div>